## Import Library

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time

## Preprocessing Data

In [ ]:
# meta 데이터 살펴보기
meta = pd.read_csv('The Movies Dataset/movies_metadata.csv',\
                     encoding='utf8',\
                     low_memory=False)

meta.head()

In [ ]:
# 추천 데이터에 필요한 컬럼만 추림
meta = meta[['id', 'original_title', 'original_language', 'genres']]

# 확인을 위해서 movie를 추가
meta = meta.rename(columns={'id' : 'movieId'})

# 영어로 된 데이터가 가장 많기 때문에, 영어로 된 데이터만 분류
meta = meta[meta['original_language'] == 'en']

meta.head()

In [ ]:
# 평점 데이터
ratings = pd.read_csv('The Movies Dataset/ratings.csv', encoding='utf8')

# 필요한 컬럼만 추출
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

In [ ]:
# check mull & data type
meta.info()

In [ ]:
# check mull & data type
ratings.info()

In [ ]:
# to_numeric은 데이터를 숫자 형식으로 바꿔주는 역할

# errors 3가지 옵션
# 1) ignore = 숫자로 변경할 수 없는 데이터라면, 원본 데이터를 그대로 반환
# 2) coerce = 숫자로 변경할 수 없는 데이터라면, 기존 데이터 삭제 후 Nan 값을 반환
# 3) raise = 숫자로 변경할 수 없는 데이터라면, 에러가 뜨면서 코드 중단

# movieId를 str -> int
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(meta.movieId, errors='coerce')

In [ ]:
# genres가 json 형태로 string으로 저장
# 이를 배열 형태로, 장르만 뽑아내서 리스트에 담아주고, 
# apply 함수를 활용해서 각 행마다 해당 함수를 적용시키게끔

def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))

    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres)

meta.head()

In [ ]:
# 데이터 합치기
data = pd.merge(ratings, meta, on='movieId', how='inner')
data.head(10)

In [ ]:
# 피벗 테이블 생성
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')
matrix.tail(20)

In [ ]:
# Model 
class NCFModel(nn.Module):
    def __init__(self, num_users, num_movies, embedding_dim=32, dropout_rate=0.5):
        super(NCFModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.movie_embedding = nn.Embedding(num_movies, embedding_dim)
        self.user_bias = nn.Embedding(num_users, 1)
        self.movie_bias = nn.Embedding(num_movies, 1)
        
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_dim * 3, 128),  # interaction 포함
            nn.BatchNorm1d(128), # 배치 정규화
            nn.ReLU(),
            nn.Dropout(dropout_rate), # 드롭아웃 적용
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, user_ids, movie_ids):
        user_embeds = self.user_embedding(user_ids)
        movie_embeds = self.movie_embedding(movie_ids)
        user_bias = self.user_bias(user_ids).squeeze()
        movie_bias = self.movie_bias(movie_ids).squeeze()
        
        interaction = torch.mul(user_embeds, movie_embeds) # GMF (Generalized Matrix Factorization) 방식
        combined = torch.cat([user_embeds, movie_embeds, interaction], dim=1) # 단순히 임베딩을 결합하는 대신, 내적(dot product) 또는 행렬 곱(matrix multiplication)으로 사용자와 영화의 잠재 요인 상호작용을 모델링
        output = self.fc_layers(combined).squeeze()
        return output + user_bias + movie_bias


## Hyperparameters

In [ ]:
# 하이퍼 파라미터 설정
embedding_dim = 50
lr=0.0001
num_epochs = 150

## Train

In [ ]:
# +학습률 동적개선 코드 추가

# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화 (모델을 GPU로 이동)
model = NCFModel(num_users, num_movies).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 학습률 스케줄러 추가
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.05, patience=2, threshold=1e-5, verbose=True
)

# 학습 손실 저장 리스트 및 변수
train_losses = []
test_losses = []
best_loss = float('inf')  # 초기화: 매우 큰 값

# 학습 루프
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    train_epoch_loss = 0
    start_time = time.time()  # 에포크 시작 시간 기록

    for users, movies, ratings in train_loader:
        # 데이터도 GPU로 이동
        users = users.to(device)
        movies = movies.to(device)
        ratings = ratings.to(device)
        
        optimizer.zero_grad()
        predictions = model(users, movies)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
        train_epoch_loss += loss.item()
    
    avg_train_loss = train_epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # 테스트 손실 계산
    model.eval()  # 모델을 평가 모드로 설정
    test_epoch_loss = 0

    with torch.no_grad():  # 그래디언트 계산 중지
        for users, movies, ratings in test_loader:
            users = users.to(device)
            movies = movies.to(device)
            ratings = ratings.to(device)

            predictions = model(users, movies)
            loss = criterion(predictions, ratings)
            test_epoch_loss += loss.item()

    avg_test_loss = test_epoch_loss / len(test_loader)
    test_losses.append(avg_test_loss)

    # 최소 손실 모델 저장
    if avg_test_loss < best_loss:
        best_loss = avg_test_loss
        torch.save(model.state_dict(), 'best_model.pth')
    
    # 학습률 스케줄러 업데이트 (Test Loss 기준)
    scheduler.step(avg_test_loss)
    current_lr = scheduler.optimizer.param_groups[0]['lr']  # 현재 학습률 가져오기
    
    # # 10 에포크마다 모델 저장
    # if (epoch + 1) % 10 == 0:
    #     torch.save(model.state_dict(), f'model_epoch_{epoch + 1}.pth')
    
    # 에포크 시간 계산
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}, Time: {epoch_time:.2f} sec, lr: {current_lr:.15f}")

# 학습 및 테스트 손실 그래프 시각화
plt.figure(figsize=(8, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), test_losses, label='Test Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend()
plt.show()

print(f"Best model saved with Test Loss: {best_loss:.4f}")



## Inference

In [ ]:
# 영화 평가 추천
def recommend_movies(user_id, model, movie_ids, top_n=10, device='cuda'):
    model.eval()
    model = model.to(device)  #gpu
    user_tensor = torch.tensor([user_id] * len(movie_ids), dtype=torch.long).to(device)
    movie_tensor = torch.tensor(movie_ids, dtype=torch.long).to(device)
    
    with torch.no_grad():
        predictions = model(user_tensor, movie_tensor).cpu().numpy()  # GPU에서 계산 후 CPU로
    
    top_movies = predictions.argsort()[-top_n:][::-1]
    return [(movie_encoder.inverse_transform([movie_ids[i]])[0], predictions[i]) for i in top_movies]

# 추천 결과를 출력하는 함수
def display_recommendations(recommendations, data):
    print("Recommendations Movies:")
    for movie_id, score in recommendations:
        # 해당 movie_id에 맞는 데이터를 검색
        movie_info = data[data['movieId'] == movie_id]
        if not movie_info.empty:  # movie_info가 비어있지 않을 경우
            title = movie_info.iloc[0]['original_title']
            genres = ", ".join(movie_info.iloc[0]['genres'])

            print(f"Title: {title}, Genres: {genres}, Score: {score:.2f}")
        else:
            print(f"Movie ID: {movie_id}, Score: {score:.2f} (정보 없음)")


## Show Result

In [ ]:
# 특정 userId에 대한 영화 추천
user_id = 11  # userId 
all_movie_ids = range(num_movies)
recommendations = recommend_movies(user_id, model, all_movie_ids)


# 추천된 영화 정보 출력
display_recommendations(recommendations, data)


In [ ]:
# If u want to only inference Using Pre-Trained model

model.load_state_dict(torch.load('./saved_models/train&test_150epochs_lr0.0001/best_model.pth'))
model.eval()

In [ ]:
# 영화 평가 추천
def recommend_movies(user_id, model, movie_ids, top_n=10, device='cuda'):
    model.eval()
    model = model.to(device)  #gpu
    user_tensor = torch.tensor([user_id] * len(movie_ids), dtype=torch.long).to(device)
    movie_tensor = torch.tensor(movie_ids, dtype=torch.long).to(device)
    
    with torch.no_grad():
        predictions = model(user_tensor, movie_tensor).cpu().numpy()  # GPU에서 계산 후 CPU로
    
    top_movies = predictions.argsort()[-top_n:][::-1]
    return [(movie_encoder.inverse_transform([movie_ids[i]])[0], predictions[i]) for i in top_movies]

# 추천 결과를 출력하는 함수
def display_recommendations(recommendations, data):
    print("Recommendations Movies:")
    for movie_id, score in recommendations:
        # 해당 movie_id에 맞는 데이터를 검색
        movie_info = data[data['movieId'] == movie_id]
        if not movie_info.empty:  # movie_info가 비어있지 않을 경우
            title = movie_info.iloc[0]['original_title']
            genres = ", ".join(movie_info.iloc[0]['genres'])

            print(f"Title: {title}, Genres: {genres}, Score: {score:.2f}")
        else:
            print(f"Movie ID: {movie_id}, Score: {score:.2f} (정보 없음)")

In [ ]:
# 사용자 ID 설정
user_id = 29  # 예시: userId
all_movie_ids = range(num_movies)  # 모든 영화의 ID 목록

# 모델을 사용하여 영화 추천
recommendations = recommend_movies(user_id, model, all_movie_ids, top_n=30)

# 추천된 영화 정보 출력
display_recommendations(recommendations, data)